### An example of Automatic Differentiation

In [1]:
using PauliPropagation

Note that we will define a lot of variables going forward as constant via the `const` syntax. In Julia, this does not fix the value of the variable, but its type. This is vital when using global variables inside functions so that performance is maintained.

In [2]:
# denote with `const` so the code that uses this global variable remains fast
const nq = 16

# denote with `const` so the code that uses this global variable remains fast
const topology = bricklayertopology(nq);

We define a transverse field Hamiltonian, whose derivative we will compute. This could be used within a variational energy minimization routine to find its ground state. 

The Hamiltonian here reads $H = \sum_{i}X_i + \sum_{\langle i, j\rangle}Z_iZ_j$ where $ \langle i, j\rangle$ denotes neighbors on the topology.

In [3]:
H = PauliSum(nq)

for qind in 1:nq
    add!(H, :X, qind, 1.0)
end

for pair in topology
    add!(H, [:Z, :Z], collect(pair), 1.0)
end

H

PauliSum(nqubits: 16, 31 Pauli terms:
 1.0 * IIXIIIIIIIIIIIII
 1.0 * IIIIIIIIIIIIXIII
 1.0 * IIIIIIIIIIIIIIIX
 1.0 * IZZIIIIIIIIIIIII
 1.0 * IIIIIIIIIIIIIXII
 1.0 * IIIIIIXIIIIIIIII
 1.0 * IIIIIIIIZZIIIIII
 1.0 * XIIIIIIIIIIIIIII
 1.0 * IIIIXIIIIIIIIIII
 1.0 * IIIIIIIIIIIIZZII
 1.0 * IIIIZZIIIIIIIIII
 1.0 * IIIIIZZIIIIIIIII
 1.0 * IIIIIIIZZIIIIIII
 1.0 * IIIIIXIIIIIIIIII
 1.0 * IIIIIIIIIIIZZIII
 1.0 * IIIIIIIIIIXIIIII
 1.0 * IIIIIIIIIZZIIIII
 1.0 * IIIZZIIIIIIIIIII
 1.0 * IIIIIIIIIIIXIIII
 1.0 * IIIXIIIIIIIIIIII
  ⋮)

Define some generic quantum circuit

In [4]:
nl = 4

# denote with `const` so the code that uses this global variable remains fast
const circuit = hardwareefficientcircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)

252

Importantly, we need to set our truncations. Depending on which package you are using to compute your gradients, you can use different truncations. 

`ReverseDiff` for example is a sophisticated package for automatic _reverse-mode_ differentiation. It will build a computational graph that it then differentiates using the chain rule. This is how large-scale neural networks are trained, and is commonly referred to as gradient _backpropagation_. The challenge here is that the graph for the chain rule is computed once (to the best of our knowledge), which means that only truncations during the initial computation will be respected. Truncations that we think work well here are `max_weight`, `max_freq`, and `max_sins`, as they do not depend on the particular parameters of the quantum circuit. On the other hand, which paths are explore with truncations such as `min_abs_coeff` will not be updated (again, to the best of our knowledge) as the gradients are computed.

Packages such as `ForwardDiff` or manual numerical differentiation, on the other hand, always involve computation of the loss function, which is affected by all truncations. Unfortunately, these methods are slower for circuits with more than several dozen parameters.

So let's wrap the coefficients into `PauliFreqTracker`, which keeps track how many times a path splits at a `PauliRotation`. We will use this to truncate our simulation, i.e., we will set a `max_freq` truncation. One could also truncate on `min_abs_coeff`, but `ReverseDiff` would not continually update which paths are truncated as you train based on which currently have small coefficient (at least we think so).

In [5]:
# the fields on PauliFreqTracker
fieldnames(PauliFreqTracker)

(:coeff, :nsins, :ncos, :freq)

The `coeff` field carries the coefficient as you are used to. But the other fields are used to keep track of additional things.

In [6]:
# an example PauliFreqTracker
# actually, PauliFreqTracker(1.0) also initializes the other fields to 0

# PauliFreqTracker(1.0)
PauliFreqTracker(1.0, 0, 0, 0)

PauliFreqTracker{Float64}(coeff=1.0, nsins=0, ncos=0, freq=0)

In [7]:
# create a PauliString with PauliFreqTracker coefficient
wrapped_pstr = PauliString(2, :X, 1, PauliFreqTracker(0.5, 0, 0, 0))

PauliString(nqubits: 2, PauliFreqTracker(0.5) * XI)

In [8]:
# showcase what this PathProperties type tracks
gate = PauliRotation([:Z, :Z], [1, 2])
θ = 0.4
wrapped_psum = propagate(gate, wrapped_pstr, θ)

PauliSum(nqubits: 2, 2 Pauli terms:
 PauliFreqTracker(-0.19471) * YZ
 PauliFreqTracker(0.46053) * XI
)

You can see that we tracked that both Pauli paths branched once (`freq=1`), and they received a `cos` or `sin` coefficient.

In [9]:
coefficients(wrapped_psum)

ValueIterator for a Dict{UInt8, PauliFreqTracker{Float64}} with 2 entries. Values:
  PauliFreqTracker{Float64}(coeff=-0.19470917115432526, nsins=1, ncos=0, freq=1)
  PauliFreqTracker{Float64}(coeff=0.46053049700144255, nsins=0, ncos=1, freq=1)

You can truncate based on these properties, which is valid even when being completely agnostic towards the parameters of the circuit.

In [10]:
# truncates everything that splits more than 0 times - this is everything
wrapped_psum = propagate(gate, wrapped_pstr, θ; max_freq = 0)

PauliSum(nqubits: 2, (no Pauli strings))

In [11]:
# truncates everything with more with nsins > 0
wrapped_psum = propagate(gate, wrapped_pstr, θ; max_sins = 0)

PauliSum(nqubits: 2, 1 Pauli term: 
 PauliFreqTracker(0.46053) * XI
)

In [12]:
# these truncations do not work with the coefficients are not properly wrapped
pstr = PauliString(2, :X, 1, 0.5)
propagate(gate, pstr, θ; max_freq = 1, max_sin = 1)

LoadError: ArgumentError: The `max_freq` and `max_sins` truncations can only be used with coefficients wrapped in `PathProperties` types.
Consider using `wrapcoefficients() with the `PauliFreqTracker` type.

Because it can be annoying to define your observable like this, we provide the function `wrapcoefficients()`, which returns a new `PauliString` or `PauliSum` where all coefficients are wrapped in the `PathProperties` type provided.

In [13]:
wrapcoefficients(pstr, PauliFreqTracker) == wrapped_pstr

true

Now do this to our Hamiltonian.

In [14]:
wrapped_H = wrapcoefficients(H, PauliFreqTracker)

PauliSum(nqubits: 16, 31 Pauli terms:
 PauliFreqTracker(1.0) * IIXIIIIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIIIIIIXIII
 PauliFreqTracker(1.0) * IIIIIIIIIIIIIIIX
 PauliFreqTracker(1.0) * IZZIIIIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIIIIIIIXII
 PauliFreqTracker(1.0) * IIIIIIXIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIIZZIIIIII
 PauliFreqTracker(1.0) * XIIIIIIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIXIIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIIIIIIZZII
 PauliFreqTracker(1.0) * IIIIZZIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIZZIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIZZIIIIIII
 PauliFreqTracker(1.0) * IIIIIXIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIIIIIZZIII
 PauliFreqTracker(1.0) * IIIIIIIIIIXIIIII
 PauliFreqTracker(1.0) * IIIIIIIIIZZIIIII
 PauliFreqTracker(1.0) * IIIZZIIIIIIIIIII
 PauliFreqTracker(1.0) * IIIIIIIIIIIXIIII
 PauliFreqTracker(1.0) * IIIXIIIIIIIIIIII
  ⋮)

Generate some generic parameters

In [15]:
using Random
Random.seed!(42)
thetas = randn(nparams);

One expectation evaluation

In [16]:
max_freq = 25
max_weight = 5

@time psum = propagate(circuit, wrapped_H, thetas; max_freq, max_weight);
overlapwithzero(psum)

  0.244301 seconds (341.63 k allocations: 22.156 MiB, 71.64% compilation time)


-1.8904666076774599

#### What does not work:

Now wrap it into a function that takes only `thetas` as argument. This is why we denoted many global variables as `const`, because we use them in here. Alternatively, one could have used so called `let` blocks for local variable namespaces.

This loss function does not work because the `ReverseDiff` package needs to propagate its custom coefficient type. But `H` is already stricktly typed. So the following loss function would not be automatically differentiable.

In [17]:
function naivelossfunction(thetas)   
    psum = propagate(circuit, wrapped_H, thetas; max_freq, max_weight);
    return overlapwithzero(psum)
end

naivelossfunction (generic function with 1 method)

In [18]:
@time naivelossfunction(thetas)

  0.314913 seconds (2.22 M allocations: 113.509 MiB, 3.94% gc time, 65.74% compilation time)


-2.1101816962371744

An example of how it would break:

In [19]:
# using Pkg; Pkg.add("ReverseDiff")
using ReverseDiff: gradient

gradient(naivelossfunction, thetas)

LoadError: ArgumentError: Converting an instance of ReverseDiff.TrackedReal{Float64, Float64, Nothing} to Float64 is not defined. Please use `ReverseDiff.value` instead.

#### What works:

We now create a loss function that does indeed work. It requires that we build the Hamiltonian with the correct coefficient type, which here is the element type of `thetas`. This will make everything differentiable.

In [20]:
function lossfunction(thetas)
    # differentiation libraries use custom types to trace through the computation
    # we need to make all of our objects typed like that so that nothing breaks
    CoeffType = eltype(thetas)

    # define H again 
    H = PauliSum(nq, CoeffType)
    for qind in 1:nq
        add!(H, :X, qind, CoeffType(1.0))
    end
    for pair in topology
        add!(H, [:Z, :Z], collect(pair), CoeffType(1.0))
    end

    # wrapp the coefficients into PauliFreqTracker so that we can use `max_freq` truncation.
    wrapped_H = wrapcoefficients(H, PauliFreqTracker)

    # be also need to run the in-place version with `!`, because by default we copy the Pauli sum
    wrapped_H = propagate!(circuit, wrapped_H, thetas; max_freq, max_weight);
    return overlapwithzero(wrapped_H)
end

lossfunction (generic function with 1 method)

Instead, we need to define a loss function that creates H every time with the correct coefficient type:

In [21]:
@time lossfunction(thetas)

  0.110272 seconds (206.92 k allocations: 16.073 MiB, 37.94% compilation time)


-1.8904666076774599

And gradients work.

In [22]:
@time gradient(lossfunction, thetas)

  1.991867 seconds (19.70 M allocations: 867.953 MiB, 12.76% gc time, 45.89% compilation time)


252-element Vector{Float64}:
 -0.5057064083446595
  0.025028009253606664
 -0.47337187596655284
  0.09285919084901251
  0.0900343739787604
  0.26751480771265024
  0.07722722421907667
  0.5186194278104864
 -0.4367624010611486
  0.7767471716936789
  0.0227644087188041
  0.908680529040292
 -0.11929346203144106
  ⋮
 -0.4835428738051231
 -0.2487625833982674
 -0.7705492109786312
  0.2651823350777113
 -1.2311240255419973
  1.4595405277023366
  0.3972920774850484
 -0.7513152897757447
 -1.0433085832333235
  0.6822884973623046
 -0.4331718877680776
 -0.24910861064112663

Now import ReverseDiff and follow their example for very fast gradients:

In [23]:
using ReverseDiff: GradientTape, gradient!, compile

In [24]:
### This is following an ReverseDiff.jl example

# some inputs and work buffer to play around with
grad_array = similar(thetas);

# pre-record a GradientTape for `gradsimulation` using inputs of length m with Float64 elements
@time const simulation_tape = GradientTape(lossfunction, thetas)

# first evaluation compiles and is slower
@time gradient!(grad_array, simulation_tape, thetas)
# second evaluation
@time gradient!(grad_array, simulation_tape, thetas);

  1.005530 seconds (17.09 M allocations: 739.427 MiB, 33.08% gc time, 0.52% compilation time)
  0.410735 seconds (238.85 k allocations: 11.782 MiB, 26.70% compilation time)
  0.310207 seconds


In [25]:
# compile to make it even faster
@time const compiled_simulation_tape = compile(simulation_tape)

# some inputs and work buffer to play around with
grad_array_compiled = similar(thetas);

# first evaluation compiles and is slower
@time gradient!(grad_array_compiled, compiled_simulation_tape, thetas)
# second evaluation
@time gradient!(grad_array_compiled, compiled_simulation_tape, thetas);

  1.960937 seconds (27.74 M allocations: 1.155 GiB, 14.67% gc time, 14.14% compilation time)
  0.272109 seconds (74.84 k allocations: 3.674 MiB, 21.19% compilation time)
  0.215754 seconds


`grad_array` here carries the gradient result. It is changed in-place in `gradient!` so that the array does not need to get allocated over and over.

See how calculating the gradient is only a few times slower than calculating the loss! The magic if reverse-mode differentiation. Feel free to explore other automatic differentiation libraries, including ones using forward-mode differentiation for when you have very few parameters. Also keep in mind that the loss functions we have defined can be sped up by not either declaring the global variables as `const` or by passing them via so-called *closures*.